In [1]:
import pandas as pd
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.phrases import Phrases, Phraser

from constants import FULL_PATH, NLP_PATH, POS, NOUN, PROPN, TOKEN, HASH, SENT_IDX

In [2]:
print(NLP_PATH)
df = pd.read_pickle(NLP_PATH + '/OnlineParticipation_nlp.pickle')
df_small = df[df[POS].isin({NOUN, PROPN})]

../../master_cloud/corpora/preprocessed/nlp


In [3]:
df_small.head(10)

,hash,index,sent_idx,text,token,POS,ent_iob,noun_phrase
0,492819066359988780,0,0,Nebentätigkeiten,Nebentätigkeit,NOUN,O,0
2,492819066359988780,2,0,OB,OB,NOUN,B,0
4,492819066359988780,4,0,Kommunalpoilter,Kommunalpoilter,NOUN,B,0
7,492819066359988780,7,0,Ehrenamt,Ehrenamt,NOUN,O,1
13,492819066359988780,13,1,Ehrenamt,Ehrenamt,NOUN,O,2
18,492819066359988780,18,1,Komunalpolitiker,Komunalpolitiker,NOUN,O,0
20,492819066359988780,20,1,OB`s,OB`s,PROPN,O,0
24,492819066359988780,24,1,Verwaltungsräten,Verwaltungsrat,NOUN,O,0
26,492819066359988780,26,1,Aufsichtsräten,Aufsichtsrat,NOUN,O,0
28,492819066359988780,28,1,Geld,Geld,NOUN,O,0


***TODO:*** *extract and add phrases from noun-chunks*

In [5]:
def docs_to_lists(token_series):
    return tuple(token_series.tolist())

df_new = df_small.groupby([HASH])[TOKEN].agg(docs_to_lists)
df_new[:10]

hash
-9221491712664867521    (Parkplatz, Parkplatz, Car-Sharing, Autofahrer...
-9220701987360669556    (Beitrag, lieber, Gast, Dank, Beitrag, Thema, ...
-9220622961416172005    (Beitrag, Hallo, Klaus, Geich-Gimbel, Dank, Be...
-9220584449090989555    (Ausgestaltung, Durchgangstr, Hotel, Mauer, Ge...
-9220276329670491563    (Leute, Leute, Auto, Müll, Grünanlage, Autobah...
-9220091770673979750    (Lanxess, Arena, Verkehrssituation, Veranstalt...
-9220043738131129346    (Re, Ort, Skulptur, Köln, Zufall, Kunstobjekt,...
-9219858721147133068          (Winterzeit, Laubbeseitigung, Winterdienst)
-9218570044050632273    (Hallo, Urbacher, Hallo, Urbacher, Dank, Komme...
-9217255457955317799           (Bürgerinitiative, Mülheimer, Müll, Frage)
Name: token, dtype: object

In [6]:
texts = df_new.values

In [7]:
# phrase detection -> results are suboptimal, even when performed on the full corpus
phrases = gensim.models.phrases.Phrases(
    texts, 
    min_count=3, 
    threshold=10,
    # common_terms=stopwords, 
    # scoring='npmi'
)
bigram = gensim.models.phrases.Phraser(phrases)
[bigram[line] for line in texts][:10]

[['Parkplatz_Parkplatz',
  'Car-Sharing',
  'Autofahrerin',
  'Parkplatz',
  'Kreis_Stadt'],
 ['Beitrag_lieber',
  'Gast_Dank',
  'Beitrag_Thema',
  'Vorschlag_Grund',
  'Übersichtlichkeit_Vorschlag',
  'Verständnis_Dank',
  'Moderation_Nowzamanus'],
 ['Beitrag_Hallo',
  'Klaus',
  'Geich-Gimbel',
  'Dank_Beitrag',
  'Vorschlag_Kategorie',
  'Radwegqualität_Thema',
  'Verständnis_Kategorie',
  'Beschilderung_Markierung',
  'Vorschlag_Auswertung',
  'Themenbereich_Beitrag',
  'Kategorie_Verständnis',
  'Dank_Moderation',
  'Wähner'],
 ['Ausgestaltung',
  'Durchgangstr',
  'Hotel',
  'Mauer',
  'Gelände',
  'Grünanlage',
  'Hotel',
  'Art',
  'Lärmschutzwall',
  'Gelände'],
 ['Leute_Leute', 'Auto', 'Müll_Grünanlage', 'Autobahnauffahrt', 'Militärring'],
 ['Lanxess_Arena',
  'Verkehrssituation',
  'Veranstaltung',
  'Lanxess_Arena',
  'Anwohner',
  'Deutz',
  'Großveranstaltung',
  'Kölnmarathon',
  'Kirmes',
  'Anwohnerschutzbereich',
  'Rheinenergie_Stadion',
  'Verkehrsplaner'],
 ['Re_O

In [9]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [14]:
ldamodel = LdaModel(corpus=corpus, num_topics=50, id2word=dictionary, passes=2)

In [20]:
len(ldamodel.show_topics())

10